In [14]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.static import teams
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import shap
from tqdm import tqdm

# Regressore logistico
### Tento di prevedere il risultato della partita basandomi solo sui precedenti di quelle due squadre

In [15]:
# Estraggo TUTTE LE PARTITE DELLE ULTIME STAGIONI delle due squadre di cui voglio fare la previsione
def match_A_B(team_id1, team_id2):
    final_df = pd.DataFrame()
    for i in range(1,4):    # stagioni da 2021-2022 fino a 2023-2024 (3)
        season = f"202{i}-2{i+1}"

        # Ottieni i dettagli delle partite giocate nella stagione specificata
        game_log = leaguegamelog.LeagueGameLog(season=season)
        
        # Ottieni i dati dei DataFrame
        game_log_data = game_log.get_data_frames()[0]
        
        # Stampa i dettagli delle partite giocate nella stagione specificata
        game_log_data_A = game_log_data[game_log_data['TEAM_ID'] == team_id1]
        game_log_data_B = game_log_data[game_log_data['TEAM_ID'] == team_id2]
        
        final_df = pd.concat([final_df, game_log_data_A, game_log_data_B], axis = 0, ignore_index=True)
    return final_df

final_df = match_A_B(teams.find_team_by_abbreviation('lal')['id'], teams.find_team_by_abbreviation('bos')['id'])

final_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22021,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19,LAL vs. GSW,L,240,45,...,40,45,21,7,4,18,25,114,-7,1
1,22021,1610612747,LAL,Los Angeles Lakers,0022100025,2021-10-22,LAL vs. PHX,L,240,34,...,34,46,25,7,3,16,26,105,-10,1
2,22021,1610612747,LAL,Los Angeles Lakers,0022100040,2021-10-24,LAL vs. MEM,W,240,45,...,27,36,29,10,12,17,14,121,3,1
3,22021,1610612747,LAL,Los Angeles Lakers,0022100053,2021-10-26,LAL @ SAS,W,265,50,...,47,56,20,8,10,13,21,125,4,1
4,22021,1610612747,LAL,Los Angeles Lakers,0022100061,2021-10-27,LAL @ OKC,L,240,46,...,30,40,29,8,2,17,22,115,-8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,22023,1610612738,BOS,Boston Celtics,0022301023,2024-03-23,BOS @ CHI,W,240,44,...,29,41,28,5,3,12,9,124,11,1
475,22023,1610612738,BOS,Boston Celtics,0022301034,2024-03-25,BOS @ ATL,L,240,47,...,26,38,21,7,8,9,16,118,-2,1
476,22023,1610612738,BOS,Boston Celtics,0022301060,2024-03-28,BOS @ ATL,L,265,45,...,34,43,28,7,6,12,15,122,-1,1
477,22023,1610612738,BOS,Boston Celtics,0022301074,2024-03-30,BOS @ NOP,W,240,39,...,43,51,27,7,7,13,13,104,12,1


In [16]:
# Prevedo quali statistiche può fare una certa squadra in base al suo id (cosidero solo partite dell'ultima stagione)
def predict_stats(id):
    season = '2023-24'
 
    # Ottieni i dettagli delle partite giocate nella stagione specificata
    game_log = leaguegamelog.LeagueGameLog(season=season)
    
    # Ottieni i dati dei DataFrame
    game_log_data = game_log.get_data_frames()[0]

    # Dividi il dataset in features (variabili indipendenti) e target (variabili dipendenti)
    X = game_log_data[['TEAM_ID']]
    y = game_log_data[['PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']]

    # Addestramento del modello di regressione lineare multi-output
    model = MultiOutputRegressor(LinearRegression())
    model.fit(X, y)

    # Fare le previsioni
    predicted_stats = model.predict([[id]])

    return predicted_stats

# A partite dalle statistiche previste precedentemente, tento di prevedere vittoria o sconfitta
def get_regressor_from_stats(df):
    X = df[['PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']]  # Seleziona le caratteristiche rilevanti
    y = df['WL']  # Etichetta di output

    # Dividi i dati in set di addestramento e set di test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Inizializza il modello di regressione logistica
    model = LogisticRegression(max_iter=1000)

    # Addestra il modello
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    print("Accuracy = {:.2}".format(accuracy_score(y_test, predictions)))

    return model


# ---- MAIN ----

# Estraggo id delle squadre
team_id1 = teams.find_team_by_abbreviation('was')['id'] # Boston Celtics
team_id2 = teams.find_team_by_abbreviation('bos')['id'] # Washington Wizards

m = get_regressor_from_stats(match_A_B(team_id1, team_id2))

# Prevedo statistiche di squadra A
stats = predict_stats(team_id1)
# Prevedo WL di squadra A
print(m.predict(stats))
# Prevedo statistiche di squadra B
stats = predict_stats(team_id2)
# Prevedo WL di squadra B
print(m.predict(stats))

#  ---- ATTENZIONE CHE LE PREDIZIONI SONO INDIPENDENTI: NON è GARANTITA MUTUA ESCLUSIONE DEGLI ESITI ----

Accuracy = 0.79
['L']
['W']


# Predizzione basata sullo stato di forma della squadra
### Metodo per calcolare la forza di una squadra

In [17]:
def ft_FPW_player(player, boxscore_data_cleaned):
    # Calcola i fantasy points utilizzando le statistiche disponibili e i coefficienti appropriati
    coeff_pts = 1.0
    coeff_reb = 1.2
    coeff_ast = 1.5
    coeff_stl = 3.0
    coeff_blk = 3.0
    coeff_to = -1.0
    
    player_stats=boxscore_data_cleaned[boxscore_data_cleaned["PLAYER_ID"] == player]

    fantasy_points = (player_stats['PTS'] * coeff_pts +
                    player_stats['REB'] * coeff_reb +
                    player_stats['AST'] * coeff_ast +
                    player_stats['STL'] * coeff_stl +
                    player_stats['BLK'] * coeff_blk +
                    player_stats['TO'] * coeff_to)
    # print(player)
    # print(fantasy_points)
    # print(player_stats['MIN'].to_list()[0])
    
    minutes = float(f"{player_stats['MIN'].to_list()[0].split('.')[0]}.{player_stats['MIN'].to_list()[0].split(':')[-1]}")
    #print(fantasy_points*minutes)
    ##pesato in base ai minuti giocati
    #print(f"minutes: {minutes}, fp: {float(fantasy_points)}")
    fantasy_points_w = float(fantasy_points) * minutes
    #print("fw: ", fantasy_points_w)
    ## ritorno solo il FP dell'ultima season
    return fantasy_points_w

# Ottengo FP del team pesato
def get_FP_team_weigthed(team_players_id, boxscore_data_cleaned):
    fp_team = 0

    for player in team_players_id:
        fp_team += ft_FPW_player(player, boxscore_data_cleaned)
    return fp_team

In [18]:
# MOMENTANEAMENTE INUTILE
def normalize_stats_with_weights(team_stats, weights):
    max_stats = np.max(team_stats)  # Calcola il massimo valore per ogni statistica
    weighted_stats = team_stats * weights  # Applica pesi alle statistiche
    normalized_stats = (weighted_stats / max_stats) * 100  # Normalizza le statistiche in base al massimo valore e scala da 0 a 100
    return normalized_stats

def recent_team_strength_with_weights(team_stats, weights):
    avg_stats = np.mean(team_stats, axis=0)  # Calcola le medie delle statistiche recenti
    mms = MinMaxScaler()
    avg_stats = mms.fit_transform(pd.DataFrame(avg_stats)) * weights
    #normalized_avg_stats = normalize_stats_with_weights(avg_stats, weights)  # Normalizza le statistiche con pesi
    team_strength = np.mean(avg_stats)  # Calcola la media delle statistiche normalizzate come indice di forza relativa
    return team_strength

# Calcola la forza della squadra a partire dall'abbreviazione e enenendo conto solo delle ultime n_partite partite
def get_strength_by_abv(abv, weight, n_partite):
    # Ottieni i dati delle partite passate della squadra specificata
    game_log = teamgamelog.TeamGameLog(teams.find_team_by_abbreviation(abv)['id'])

    # Ottieni i dati dei DataFrame e tengo solo le ultime k partite
    game_log_data = game_log.get_data_frames()[0][:n_partite]

    #game_log_data = game_log_data[['PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK','W_PCT']]

    # CALCOLO FP PER OGNI PARTITA
    # fp = []
    # for index, row in tqdm(game_log_data.iterrows()):
    #     boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(row["Game_ID"])
 
    #     # Ottieni i dati dei DataFrame
    #     boxscore_data_cleaned = boxscore.get_data_frames()[0].dropna()

    #     team_players_id = boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_ID"].to_list()
    #     fp.append(get_FP_team_weigthed(team_players_id, boxscore_data_cleaned))
    # game_log_data['FP'] = fp

    # Peso il risultato anche in base alla percentuale di vittorie in questa stagione
    return recent_team_strength_with_weights(game_log_data[['PTS','FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']], weight) * game_log_data['W_PCT'][0]

# Test
cle_strength = get_strength_by_abv('bos',[1,1,1,1,1,1,1,1,1],20)
print("Forza relativa della squadra nelle ultime {} partite: {:.2f}".format(20, cle_strength))

Forza relativa della squadra nelle ultime 20 partite: 0.15


## SHAP

### IGNORARE (usare la versione 2)

In [19]:
# Ottengo un dataset con una riga per ogni partita giocata nelle ultime stagioni,
# con tutte le statistiche della squadra in casa e tutte quelle di quella in trasferta
def season_games():
    final_df = pd.DataFrame()
    for i in range(1,4):
        season = f"202{i}-2{i+1}"

        # Ottieni i dettagli delle partite giocate nella stagione specificata
        game_log = leaguegamelog.LeagueGameLog(season=season)
        
        # Ottieni i dati dei DataFrame
        game_log_data = game_log.get_data_frames()[0]
        
        final_df = pd.concat([final_df, game_log_data], axis = 0)

    final_df.index = range(0, len(final_df))

    home_df = final_df[final_df['MATCHUP'].str.contains('vs')][['TEAM_ABBREVIATION','WL','PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']]
    home_df.index = range(0, int(len(final_df)/2))

    away_df = final_df[final_df['MATCHUP'].str.contains('@')][['TEAM_ABBREVIATION','PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']]
    away_df.index = range(0, int(len(final_df)/2))

    previous_games = home_df.join(away_df, lsuffix='_1', rsuffix='_2')

    return previous_games

### IGNORARE (usare la versione 2)

In [20]:
def get_regressor(df):
    mms = MinMaxScaler()
    X = df[['PTS_1','FG_PCT_1', 'FT_PCT_1', 'OREB_1','DREB_1','AST_1','FG3_PCT_1','STL_1','BLK_1',
            'PTS_2','FG_PCT_2', 'FT_PCT_2', 'OREB_2','DREB_2','AST_2','FG3_PCT_2','STL_2','BLK_2']]  # Seleziona le caratteristiche rilevanti
    y = df['WL']  # Etichetta di output
    
    X = pd.DataFrame(mms.fit_transform(X), columns = X.columns)

    # Dividi i dati in set di addestramento e set di test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inizializza il modello di regressione logistica
    model = LogisticRegression()

    # Addestra il modello
    model.fit(X_train, y_train)

    explainer = shap.Explainer(model, X)
    shap_values = explainer(X)

    weight = []
    weights = pd.DataFrame(shap_values.values, columns = X.columns)
    for col in weights.columns:
        weight.append(np.mean(np.abs(weights[col])))

    predictions = model.predict(X_test)
    print("Accuracy = {:.2}".format(accuracy_score(y_test, predictions)))

    return model, weight, weights

m, weight, weights = get_regressor(season_games())

# Test
bos_strength = get_strength_by_abv('bos',weight[:8],20)
det_strength = get_strength_by_abv('mil',weight[8:],20)
print("Forza relativa della squadra nelle ultime {} partite: {:.5f}".format(20, bos_strength))
print("Forza relativa della squadra nelle ultime {} partite: {:.5f}".format(20, det_strength))
np.abs(weights).mean(axis = 0)

Accuracy = 0.82
Forza relativa della squadra nelle ultime 20 partite: 0.05905
Forza relativa della squadra nelle ultime 20 partite: 0.00945


PTS_1        0.395490
FG_PCT_1     0.746124
FT_PCT_1     0.170538
OREB_1       0.213586
DREB_1       0.876430
AST_1        0.059871
FG3_PCT_1    0.340668
STL_1        0.383230
BLK_1        0.114694
PTS_2        0.321031
FG_PCT_2     0.033615
FT_PCT_2     0.001746
OREB_2       0.041457
DREB_2       0.042665
AST_2        0.019138
FG3_PCT_2    0.063016
STL_2        0.099758
BLK_2        0.054824
dtype: float64

### VERSIONE CORRETTA

In [21]:
# Ottengo un dataset con una riga per ogni partita di una squadra con tutte le statistiche di tale squadra
# - si ignora "in casa" e "fuori casa"
def season_games2():
    final_df = pd.DataFrame()
    for i in range(3,4):
        season = f"202{i}-2{i+1}"

        # Ottieni i dettagli delle partite giocate nella stagione specificata
        game_log = leaguegamelog.LeagueGameLog(season=season)
        
        # Ottieni i dati dei DataFrame
        game_log_data = game_log.get_data_frames()[0]
        
        final_df = pd.concat([final_df, game_log_data], axis = 0)

    final_df.index = range(0, len(final_df))

    # final_df = final_df[:10]

    #CALCOLO FP PER OGNI PARTITA (OLTRE 2000)
    fp = []
    for index, row in tqdm(final_df.iterrows()):
        boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(row["GAME_ID"])
 
        # Ottieni i dati dei DataFrame
        boxscore_data_cleaned = boxscore.get_data_frames()[0].dropna()

        team_players_id = boxscore_data_cleaned[boxscore_data_cleaned["TEAM_ABBREVIATION"] == row['MATCHUP'].split(" ")[0]]["PLAYER_ID"].to_list()
        fp.append(get_FP_team_weigthed(team_players_id, boxscore_data_cleaned))
    final_df['FP'] = fp

    return final_df
final_df = season_games2()[['TEAM_ABBREVIATION','WL','PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK', 'FP']]
final_df

2260it [17:13,  2.19it/s]


,TEAM_ABBREVIATION,WL,PTS,FG_PCT,FT_PCT,OREB,DREB,AST,FG3_PCT,STL,BLK,FP
0,LAL,L,107,0.456,0.750,13,31,23,0.345,5,4,5995.781
1,DEN,W,119,0.527,0.750,9,33,29,0.412,9,6,7866.046
2,PHX,W,108,0.442,0.765,17,43,23,0.333,5,7,6597.447
3,GSW,L,104,0.356,0.786,18,31,19,0.233,11,6,6177.712
4,LAC,W,123,0.522,0.813,14,31,36,0.471,11,7,6413.880
...,...,...,...,...,...,...,...,...,...,...,...,...
2255,LAC,L,95,0.408,0.815,9,29,18,0.407,6,1,4530.712
2256,SAC,W,109,0.424,0.760,20,36,29,0.286,10,6,7133.214
2257,GSW,W,104,0.456,1.000,11,41,29,0.469,9,4,6216.527
2258,DAL,L,100,0.438,0.636,8,34,27,0.375,8,7,6860.128


### VERSIONE CORRETTA

In [25]:
def get_regressor2(df):
    mms = MinMaxScaler()
    X = df[['PTS','FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK']]  # Seleziona le caratteristiche rilevanti
    y = df['WL']  # Etichetta di output
    
    X = pd.DataFrame(mms.fit_transform(X), columns = X.columns)

    # Dividi i dati in set di addestramento e set di test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inizializza il modello di regressione logistica
    model = LogisticRegression()

    # Addestra il modello
    model.fit(X_train, y_train)

    explainer = shap.Explainer(model, X)
    shap_values = explainer(X)

    weight = []
    weight_tab = pd.DataFrame(shap_values.values, columns = X.columns)
    for col in weight_tab.columns:
        weight.append(np.mean(np.abs(weight_tab[col])))

    predictions = model.predict(X_test)
    print("Accuracy = {:.2}".format(accuracy_score(y_test, predictions)))

    return model, weight, weight_tab

m, weight, weight_tab = get_regressor2(final_df)

Accuracy = 0.85


In [26]:
# Controllo i pesi
np.abs(weight_tab).mean(axis = 0)

PTS        0.353532
FG_PCT     0.677303
FT_PCT     0.163215
OREB       0.177474
DREB       0.797015
AST        0.028063
FG3_PCT    0.335253
STL        0.362689
BLK        0.192061
dtype: float64

In [56]:
# Test
# ---- MAIN ----
#final_df = season_games2()[['TEAM_ABBREVIATION','WL','PTS', 'FG_PCT', 'FT_PCT', 'OREB','DREB','AST','FG3_PCT','STL','BLK','FP']]
m, weight, weight_tab = get_regressor2(final_df)

Accuracy = 0.85


In [60]:
weight

[0.35353210943121816,
 0.6773026060484189,
 0.1632154672151238,
 0.1774744792594547,
 0.7970150952121257,
 0.028062570626348697,
 0.3352530622990414,
 0.36268890394761993,
 0.1920613673696502]

In [61]:
bos_strength = get_strength_by_abv('bos',weight,50) # Calcolerà 50 FP
det_strength = get_strength_by_abv('chi',weight,50) # Calcolerà 50 FP
print("Forza relativa della squadra nelle ultime {} partite: {:.5f}".format(50, bos_strength))
print("Forza relativa della squadra nelle ultime {} partite: {:.5f}".format(50, det_strength))

Forza relativa della squadra nelle ultime 50 partite: 0.05114
Forza relativa della squadra nelle ultime 50 partite: 0.03102


# TEST

**Stagione Corrente**

In [50]:
for i in range(3,4):
    season = f"202{i}-2{i+1}"
    
    # Ottieni i dettagli delle partite giocate nella stagione specificata
    game_log = leaguegamelog.LeagueGameLog(season=season)
    
    # Ottieni i dati dei DataFrame
    game_log_data = game_log.get_data_frames()[0]

In [65]:
predictions = []
for index, row in tqdm(game_log_data.iterrows()):
    #print(f"{row['TEAM_ABBREVIATION']} vs {row['MATCHUP'].split(' ')[-1]}")
    home_strength = get_strength_by_abv(row["TEAM_ABBREVIATION"],weight,50) # Calcolerà 50 FP
    away_strength = get_strength_by_abv(row["MATCHUP"].split(" ")[-1],weight,50) # Calcolerà 50 FP
    if home_strength >= away_strength:
        predictions.append("W")
    else:
        predictions.append("L")
    
predictions

273it [03:54,  1.16it/s]


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
from sklearn.metrics import accuracy_score
print(f"Accuracy: {accuracy_score(y_true, y_pred)*100}")

**Stagioni 2022-2023 e 2023-2024**

In [45]:
predictions = []
for i in range(2,4):
    season = f"202{i}-2{i+1}"
    
    # Ottieni i dettagli delle partite giocate nella stagione specificata
    game_log = leaguegamelog.LeagueGameLog(season=season)
    
    # Ottieni i dati dei DataFrame
    game_log_data = game_log.get_data_frames()[0]
    
    for index, row in tqdm(game_log_data.iterrows()):
        home_strength = get_strength_by_abv(row["TEAM_ABBREVIATION"],weight,50) # Calcolerà 50 FP
        away_strength = get_strength_by_abv(row["MATCHUP"].split(" ")[-1],weight,50) # Calcolerà 50 FP
        if home_strength >= away_strength:
            predictions.append("W")
        else:
            predictions.append("L")

predictions

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1610612747,LAL,Los Angeles Lakers,0022300061,2023-10-24,LAL @ DEN,L,240,41,...,31,44,23,5,4,12,18,107,-12,1
1,22023,1610612743,DEN,Denver Nuggets,0022300061,2023-10-24,DEN vs. LAL,W,240,48,...,33,42,29,9,6,12,15,119,12,1
2,22023,1610612756,PHX,Phoenix Suns,0022300062,2023-10-24,PHX @ GSW,W,240,42,...,43,60,23,5,7,19,22,108,4,1
3,22023,1610612744,GSW,Golden State Warriors,0022300062,2023-10-24,GSW vs. PHX,L,240,36,...,31,49,19,11,6,11,23,104,-4,1
4,22023,1610612746,LAC,LA Clippers,0022300074,2023-10-25,LAC vs. POR,W,240,47,...,31,45,36,11,7,18,20,123,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,22023,1610612758,SAC,Sacramento Kings,0022301112,2024-04-04,SAC @ NYK,L,240,42,...,26,35,26,6,3,16,23,109,-11,1
2284,22023,1610612744,GSW,Golden State Warriors,0022301113,2024-04-04,GSW @ HOU,W,240,47,...,31,43,29,6,2,21,23,133,23,1
2285,22023,1610612745,HOU,Houston Rockets,0022301113,2024-04-04,HOU vs. GSW,L,240,36,...,17,29,23,11,1,12,19,110,-23,1
2286,22023,1610612746,LAC,LA Clippers,0022301114,2024-04-04,LAC vs. DEN,W,240,34,...,41,48,22,10,2,7,20,102,2,1
